# Solving IBM's Quantum Open Science Prize



## Piece 1: Initialize the simulators

- We use Qiskit's `QasmSimulator` to produce noise-free measurements.
- We use the `jakarta` backend to run the simulation of a Heisenberg model Hamiltonian for a three-particle system using Trotterization.


In [1]:
# IBM account 
from qiskit.providers.aer import QasmSimulator

# load IBMQ Account data
from qiskit import IBMQ

# Noiseless simulated backend
sim = QasmSimulator()

# replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
account = IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')


## Piece 2: Generate circuits

- classically simulatable (short with few qubits) circuits
- the full trotterized simulation
- customizable number of trotter steps


In [2]:
# Trotterized circuit from IBM material
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def get_circuit(measure, trotter_steps, X=True, Y=True, Z=True):

    # YOUR TROTTERIZATION GOES HERE -- START (beginning of example)

    # Parameterize variable t to be evaluated at t=pi later
    t = Parameter('t')

    # Build a subcircuit for XX(t) two-qubit gate
    XX_qr = QuantumRegister(2)
    XX_qc = QuantumCircuit(XX_qr, name='XX')

    XX_qc.ry(np.pi/2,[0,1])
    XX_qc.cnot(0,1)
    XX_qc.rz(2 * t, 1)
    XX_qc.cnot(0,1)
    XX_qc.ry(-np.pi/2,[0,1])

    # Convert custom quantum circuit into a gate
    XX = XX_qc.to_instruction()

    # Build a subcircuit for YY(t) two-qubit gate
    YY_qr = QuantumRegister(2)
    YY_qc = QuantumCircuit(YY_qr, name='YY')

    YY_qc.rx(np.pi/2,[0,1])
    YY_qc.cnot(0,1)
    YY_qc.rz(2 * t, 1)
    YY_qc.cnot(0,1)
    YY_qc.rx(-np.pi/2,[0,1])

    # Convert custom quantum circuit into a gate
    YY = YY_qc.to_instruction()

    # Build a subcircuit for ZZ(t) two-qubit gate
    ZZ_qr = QuantumRegister(2)
    ZZ_qc = QuantumCircuit(ZZ_qr, name='ZZ')

    ZZ_qc.cnot(0,1)
    ZZ_qc.rz(2 * t, 1)
    ZZ_qc.cnot(0,1)

    # Convert custom quantum circuit into a gate
    ZZ = ZZ_qc.to_instruction()

    # Combine subcircuits into a single multiqubit gate representing a single trotter step
    num_qubits = 3

    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

    for i in range(0, num_qubits - 1):
        if Z:
            Trot_qc.append(ZZ, [Trot_qr[i], Trot_qr[i+1]])
        
        if Y:
            Trot_qc.append(YY, [Trot_qr[i], Trot_qr[i+1]])
        
        if X:
            Trot_qc.append(XX, [Trot_qr[i], Trot_qr[i+1]])

    # Convert custom quantum circuit into a gate
    Trot_gate = Trot_qc.to_instruction()
    
    # YOUR TROTTERIZATION GOES HERE -- FINISH (end of example)


    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    #trotter_steps = 8  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    qr = QuantumRegister(7)
    cr = ClassicalRegister(3)
    qc = QuantumCircuit(qr, cr) if measure is True else QuantumCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        qc.append(Trot_gate, [qr[1], qr[3], qr[5]])
        if not X or not Y or not Z:
            break
    
        

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.bind_parameters({t: target_time/trotter_steps})

    if measure:
        qc.measure([1,3,5], cr)

    
    return qc

## Piece 3: Qiskit Measurement Result Wrapper

- Wrapper for Qiskit's measurement results whose counts can be overwritten


In [3]:
from qiskit.result import Result

from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.exceptions import QiskitError
from qiskit.result.counts import Counts

class OwnResult(Result):
    
    def __init__(self, result):
        self._result = result
        self._counts = {}
            
        
    def get_counts(self, experiment=None):

        if experiment is None:
            exp_keys = range(len(self._result.results))
        else:
            exp_keys = [experiment]
        

        dict_list = []
        for key in exp_keys:
            exp = self._result._get_experiment(key)
            try:
                header = exp.header.to_dict()
            except (AttributeError, QiskitError):  # header is not available
                header = None

            if "counts" in self._result.data(key).keys():
                if header:
                    counts_header = {
                        k: v
                        for k, v in header.items()
                        if k in {"time_taken", "creg_sizes", "memory_slots"}
                    }
                else:
                    counts_header = {}
                    
                    
                # CUSTOM CODE STARTS HERE #######################
                dict_list.append(Counts(
                    self._counts[str(key)] if str(key) in map(lambda k: str(k), self._counts.keys()) else self._result.data(key)["counts"]
                    , **counts_header))
                # CUSTOM CODE ENDS HERE #######################
                
            elif "statevector" in self._result.data(key).keys():
                vec = postprocess.format_statevector(self._result.data(key)["statevector"])
                dict_list.append(statevector.Statevector(vec).probabilities_dict(decimals=15))
            else:
                raise QiskitError(f'No counts for experiment "{repr(key)}"')

        # Return first item of dict_list if size is 1
        if len(dict_list) == 1:
            return dict_list[0]
        else:
            return dict_list
        
        
    def set_counts(self, counts, experiment=None):
        self._counts[str(experiment) if experiment is not None else "0"] = counts

## Piece 4: State Tomography Mitigation

- post-processing of the measured counts using the trained 

In [4]:
from qiskit.opflow import Zero, One
from qiskit.ignis.verification.tomography import StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs, modifiers, mitigate=False):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    
    own_res = OwnResult(result)
    
    idx = 0
    
    if mitigate:
        for experiment in st_qcs:
            exp_keys = [experiment]
            for key in exp_keys:

                exp = result._get_experiment(key)                
                counts = sorted_counts(result.get_counts(key))
                mitigated = {item[0]: item[1]*modifiers[idx][i] for i, item in enumerate(counts.items())}
                
                #print("original: ", sorted_counts(result.get_counts(key)))
                #print("mitigated: ", sorted_counts(mitigated))
                #print("\n")

                own_res.set_counts(mitigated, key)
    
            idx = idx + 1 
        
    
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(own_res if mitigate else result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid



/tmp/ipykernel_672/2168099722.py:2: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import StateTomographyFitter


## Piece 5: Create the training circuits

- start the jobs without using them directly

In [5]:
# prepare the circuits
from qiskit.ignis.verification.tomography import state_tomography_circuits

# trotter steps
trotters = 8

# repititions
reps = int(trotters/2)

train_st_qcs_xy = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])
train_st_qcs_yz = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])
train_st_qcs_zx = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])



In [9]:
from qiskit import QuantumCircuit, transpile, assemble

def start_train_jobs(qc, shots = 8192):
    t_qc = transpile(qc, jakarta)
    qobj = assemble(t_qc)
    jobs = jakarta.run(qobj, shots=shots)
    return jobs


In [10]:
# The XY transition

xy_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_xy), range(reps)))

for job in xy_jobs:
    print("'{}',".format(job.job_id()))

/tmp/ipykernel_672/2460616565.py:6: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  jobs = jakarta.run(qobj, shots=shots)


'625026e173968c63fd07bdf7',
'625026e3182d02129f4c353a',
'625026e5caa2654b46f1a54c',
'625026e7caa2650a98f1a54d',


In [11]:
# The YZ transition
yz_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_yz), range(reps)))

for job in yz_jobs:
    print("'{}',".format(job.job_id()))

'625026ed182d021bea4c353d',
'625026efaacb9b36e35f5880',
'625026f0caa2657e4bf1a54e',
'625026f373968c246007bdf9',


In [12]:
# The ZX transition

zx_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_zx), range(reps)))

for job in zx_jobs:
    print("'{}',".format(job.job_id()))

'625026f44b5152e64c7c7689',
'625026f6182d02ac254c353e',
'625026f8d7203323c267e36b',
'625026facfe45c8111e5ae7b',


## Piece 6: Run the evaluation circuits

In [13]:
st_qcs = state_tomography_circuits(get_circuit(False, trotters, True, True, True), [1,3,5])
eval_jobs = list(map(lambda x: start_train_jobs(st_qcs), range(8)))

for job in eval_jobs:
    print("'{}',".format(job.job_id()))

'62502702a5d4eea76e77d85a',
'6250270973968c09e107bdfa',
'62502710a5d4ee34d277d85b',
'625027185ab79b6a72f95d46',
'62502720d7203307a967e36e',
'62502727d72033370867e370',
'6250272fd7203362ae67e371',
'62502736a5d4ee564177d85c',


In [7]:
job_ids_xy = [
    '625026e173968c63fd07bdf7',
    '625026e3182d02129f4c353a',
    '625026e5caa2654b46f1a54c',
    '625026e7caa2650a98f1a54d'
]

In [8]:
job_ids_yz = [
    '625026ed182d021bea4c353d',
    '625026efaacb9b36e35f5880',
    '625026f0caa2657e4bf1a54e',
    '625026f373968c246007bdf9'
]

In [9]:
job_ids_zx = [
    '625026f44b5152e64c7c7689',
    '625026f6182d02ac254c353e',
    '625026f8d7203323c267e36b',
    '625026facfe45c8111e5ae7b'
]

In [17]:
complete_job_ids = [
    '62502702a5d4eea76e77d85a',
    '6250270973968c09e107bdfa',
    '62502710a5d4ee34d277d85b',
    '625027185ab79b6a72f95d46',
    '62502720d7203307a967e36e',
    '62502727d72033370867e370',
    '6250272fd7203362ae67e371',
    '62502736a5d4ee564177d85c'
]

## Piece 7: retrieve the completed results

In [10]:
jobs_xy = list(map(jakarta.retrieve_job, job_ids_xy))
jobs_yz = list(map(jakarta.retrieve_job, job_ids_yz))
jobs_zx = list(map(jakarta.retrieve_job, job_ids_zx))

## Piece 8: Calculate the modifiers

In [11]:
from collections import Counter
from functools import reduce
from qiskit import assemble, execute, transpile


def sorted_counts(counts):
    complete = dict(reduce(lambda a, b: a.update(b) or a, [{'000': 0, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0}, counts], Counter()))
    return {k: v for k, v in sorted(complete.items(), key=lambda item: item[0])}


def get_jakarta_modifiers(qc, job, shots = 8192, display=True):
    
    t_qc_sim = transpile(qc, sim)
    noiseless_result = sim.run( assemble(t_qc_sim), shots=shots).result()
    noiseless_counts = sorted_counts(noiseless_result.get_counts())
    
    #t_qc = transpile(qc, sim_noisy_jakarta)
    #qobj = assemble(t_qc)
    counts = sorted_counts(job.result().get_counts(qc))
    
    zipped = list(zip(noiseless_counts.values(), counts.values()))
    modifier = list(map(lambda pair: pair[0]/pair[1] if pair[1] > 0 else 1, zipped))

    if display is True:
        print("noisy:     ", counts)
        print("nose-free: ", noiseless_counts)

        print("modifier: ", modifier)
        print("\n")
    
    return modifier



In [ ]:

# each variable is a list of the trotters/2=6 jobs. Each job is a list of 27 state tomography circuits. Each of these lists 
list_modifiers_xy = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc,job, display=False), train_st_qcs_xy)), jobs_xy))
list_modifiers_yz = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc, job, display=False), train_st_qcs_yz)), jobs_yz))
list_modifiers_zx = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc, job, display=False), train_st_qcs_zx)), jobs_zx))

In [13]:

from functools import reduce

# reorder the data to make the
state_tomography_xy = list(zip(*list_modifiers_xy))
state_tomography_yz = list(zip(*list_modifiers_yz))
state_tomography_zx = list(zip(*list_modifiers_zx))

zipped = list(map(lambda l: [item for sublist in l  for item in sublist], list(zip(state_tomography_xy, state_tomography_yz, state_tomography_zx))))


def calc_modifier(stqc):
    mods = list(map(lambda mod_list: reduce(lambda a,b: a*b, mod_list, 1), zip(*stqc)))
    return mods
    

mods = list(map(calc_modifier,zipped))


## Piece 9: Evaluate the performance

In [18]:

shots = 1024
st_qcs = state_tomography_circuits(get_circuit(False, trotters, True, True, True), [1,3,5])


noisy_job = jakarta.retrieve_job('623d8d50a2f72db120dab8b0')
noisefree_job = execute(st_qcs, sim, shots=shots)

    
noisy_fid = state_tomo(noisy_job.result(), st_qcs, mods, mitigate=False)
noisefree_fid = state_tomo(noisefree_job.result(), st_qcs, mods, mitigate=False)
mitigated_fid = state_tomo(noisy_job.result(), st_qcs, mods, mitigate=True)


print('noisy state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean([noisy_fid]), np.std([noisy_fid])))
print('noise-free state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean([noisefree_fid]), np.std([noisefree_fid])))
print('mitigated state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean([mitigated_fid]), np.std([mitigated_fid])))


noisy state tomography fidelity = 0.1172 ± 0.0000
noise-free state tomography fidelity = 0.9619 ± 0.0000
mitigated state tomography fidelity = 0.4882 ± 0.0000


In [ ]:
unmitigated = np.mean([noisy_fid])
ideal = np.mean([noisefree_fid])
mitigated = np.mean([mitigated_fid])

error_unmitigated = abs(unmitigated-ideal)
error_mitigated = abs(mitigated-ideal)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated):", error_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal))
print("Relative error (mitigatedR):", error_mitigated/ideal)

print(f"Error reduction: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")